<a href="https://colab.research.google.com/github/doctorpeppercokebear/DL/blob/main/0213_ourmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 우리만의 모델 만들기
# 두가지 카테고리 분류 (데이터 기획은 여러분 들이


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import PIL  # python 의 이미지 전처리 라이브러리
import glob # 대용량, 대규모 파일을 다룰때 사용
import os # os, path 관련 조작, 변경 , 파일 삭제 등
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

Mounted at /content/drive


In [3]:
!mkdir dataset

In [4]:
!unzip /content/drive/MyDrive/nvidia_ai_academy/영상처리과정/stb_apl.zip -d dataset

Archive:  /content/drive/MyDrive/nvidia_ai_academy/영상처리과정/stb_apl.zip
  inflating: dataset/apple/apple-001.jpeg  
  inflating: dataset/apple/apple-002.jpeg  
  inflating: dataset/apple/apple-003.jpeg  
  inflating: dataset/apple/apple-004.jpeg  
  inflating: dataset/apple/apple-005.jpeg  
  inflating: dataset/apple/apple-006.jpeg  
  inflating: dataset/apple/apple-007.jpeg  
  inflating: dataset/apple/apple-008.jpeg  
  inflating: dataset/apple/apple-009.jpeg  
  inflating: dataset/apple/apple-010.jpeg  
  inflating: dataset/apple/apple-011.jpeg  
  inflating: dataset/apple/apple-012.jpeg  
  inflating: dataset/apple/apple-013.jpeg  
  inflating: dataset/apple/apple-014.jpeg  
  inflating: dataset/apple/apple-015.jpeg  
  inflating: dataset/apple/apple-016.jpeg  
  inflating: dataset/apple/apple-017.jpeg  
  inflating: dataset/apple/apple-018.jpeg  
 extracting: dataset/apple/apple-019.png  
  inflating: dataset/apple/apple-020.jpeg  
  inflating: dataset/apple/apple-021.jpeg 

In [5]:

dataset_folder_path = '/content/dataset'
category_cnt = len(os.listdir(dataset_folder_path))
print(os.listdir(dataset_folder_path))

['apple', 'strawberry']


In [6]:
# glob.glob(dataset_folder_path+'/*/*')
# 파일의 경로+/* 을 입력하면 해당 폴더 하위 경로 상에 있는 모든 파일을
# 리스트로 리턴
# ex) dataset_folder_path+'/*/*' -> 하위 폴더의 하위 데이터 전체 리턴
x = []
y = []

for label,folder in enumerate(os.listdir(dataset_folder_path)):

    # print(dataset_folder_path +'/'+ folder) 좋은 코드가 아님
    # os.path.join(dataset_folder_path , folder)) # 유지 보수 좋은 코드

    folder_full_path = os.path.join(dataset_folder_path , folder)
    all_files = glob.glob(folder_full_path + '/*.jpeg')
    all_files_jpg = glob.glob(folder_full_path + '/*.jpg')
    all_files_png = glob.glob(folder_full_path + '/*.png')
    all_files.extend(all_files_jpg)
    all_files.extend(all_files_png)

    for idx,file_full_path in enumerate(all_files):
        image = PIL.Image.open(file_full_path)
        image = image.resize((64,64))
        image = image.convert('RGB')
        data = np.asarray(image)
        x.append(data)
        y.append(label)
        if idx % 30 == 0:
            print(idx,'/',len(all_files))



0 / 156
30 / 156
60 / 156
90 / 156
120 / 156
150 / 156
0 / 431
30 / 431
60 / 431
90 / 431
120 / 431
150 / 431
180 / 431
210 / 431
240 / 431
270 / 431
300 / 431
330 / 431
360 / 431
390 / 431
420 / 431


In [7]:
x = np.array(x)
y = np.array(y)
print('카테고리 갯수',np.bincount(y))
print('전처리 확인',x.shape)

카테고리 갯수 [156 431]
전처리 확인 (587, 64, 64, 3)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
# 개발 팁 , 전처리 시간이 너무 오래 걸린다면(데이터 늘어 나서)
#xy = (x_train, x_test, y_train, y_test)
# np.save('/content/drive/MyDrive/preprocessing_data.npy',xy)
# x_train,x_test,y_train,y_test = np.load('/content/drive/MyDrive/preprocessing_data.npy')

In [10]:
# x_train,x_test,y_train,y_test = np.load('/content/drive/MyDrive/nvidia_ai_academy/영상처리과정.npy')

In [11]:
x_train = x_train / 255.
x_test = x_test / 255.

In [12]:
# 개선의 여지 해상도 수정 ( 전처리 도 수정 해야 함)
input_layer = layers.Input(shape = (64,64,3), name='start_layer')

# 개선의 여지 conv2d 수정 성능이 매우 잘 나온다면 -모델 다이어트
# 성능이 안나온다면 channel 수, 레이어 추가
tower_1 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)

tower_2 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_2 = layers.Conv2D(64, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2)

tower_3 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_3 = layers.Conv2D(64, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3)

tower_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(input_layer)
tower_4 = layers.Conv2D(64, kernel_size=(1,1) , padding='same' , activation='relu')(tower_4)

concat_layer = layers.concatenate([tower_1, tower_2,tower_3,tower_4], axis=3)

tower_2_1 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)

tower_2_2 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_2 = layers.Conv2D(120, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2_2)

tower_2_3 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_3 = layers.Conv2D(120, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_2_3)

tower_2_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer)
tower_2_4 = layers.Conv2D(120, kernel_size=(1,1) , padding='same' , activation='relu')(tower_2_4)

final_concat_layer = layers.concatenate([tower_2_1, tower_2_2,tower_2_3,tower_2_4], axis=3)
average_pooling_layer = layers.AveragePooling2D(pool_size=(16,16),strides=(16,16))(final_concat_layer)
flat_layer = layers.Flatten()(average_pooling_layer)

# 개선의 여지 -  Dense 층 추가
output_layer = layers.Dense(1 , activation = 'sigmoid')(flat_layer)

google_net_model = models.Model(inputs = input_layer, outputs = output_layer)
google_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start_layer (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 conv2d_3 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 3)            0         ['start_layer[0][0]']         
 D)                                                                                           

In [13]:
#개선 여지 hyper parameter tuning
# ex ) tf.keras.optimizers.Adam()
google_net_model.compile(loss='binary_crossentropy',
                         optimizer='adam', metrics=['accuracy'])

In [14]:
check_point_path = '/content/drive/MyDrive/nvidia_ai_academy/영상처리과정/ourmodel_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path,
                                        monitor='val_loss',
                                        save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)


In [15]:
google_net_model.fit(x_train,y_train,
                     batch_size=32,
                     callbacks=[mc, es],
                     epochs=20,
                     validation_data=(x_test, y_test))

Epoch 1/20
15/15 [==============================] - 24s 760ms/step - loss: 0.5923 - accuracy: 0.7228 - val_loss: 0.5648 - val_accuracy: 0.7458
Epoch 2/20
15/15 [==============================] - 4s 267ms/step - loss: 0.5625 - accuracy: 0.7313 - val_loss: 0.5307 - val_accuracy: 0.7458
Epoch 3/20
15/15 [==============================] - 2s 131ms/step - loss: 0.5441 - accuracy: 0.7271 - val_loss: 0.5443 - val_accuracy: 0.7458
Epoch 4/20
15/15 [==============================] - 4s 291ms/step - loss: 0.5290 - accuracy: 0.7463 - val_loss: 0.5043 - val_accuracy: 0.7627
Epoch 5/20
15/15 [==============================] - 2s 140ms/step - loss: 0.4906 - accuracy: 0.7804 - val_loss: 0.5135 - val_accuracy: 0.7712
Epoch 6/20
15/15 [==============================] - 2s 128ms/step - loss: 0.5131 - accuracy: 0.7569 - val_loss: 0.5318 - val_accuracy: 0.7627
Epoch 7/20
15/15 [==============================] - 2s 140ms/step - loss: 0.4647 - accuracy: 0.7910 - val_loss: 0.5345 - val_accuracy: 0.7373
Epoch

In [16]:
# ourmodel inference code
# inference만 진행 할 시 , 가장 위 import library  실행 ->  모델 빌드 하는 부분 실행

google_net_model.load_weights('/content/drive/MyDrive/nvidia_ai_academy/영상처리과정/ourmodel_checkpoint')


In [17]:
from google.colab import files
f = files.upload()

Saving strawberry-003.jpeg to strawberry-003.jpeg


In [18]:
file_name = list(f.keys())[0]
test_img = PIL.Image.open(file_name)
test_img = test_img.resize((64,64))
test_img = test_img.convert('RGB')
data = np.asarray(test_img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor / 255.
test_image_tensor.shape

(1, 64, 64, 3)

In [19]:
result = google_net_model.predict(test_image_tensor)
result[0][0]
if result[0][0] > 0.5:
    print(f'{result[0][0] * 100} % 딸기로 예상됩니다..')
else:
    print(f'{ (1 - result[0][0]) * 100} % 사과로 예측 됩니다.')

1/1 [==============================] - 1s 661ms/step
72.90275692939758 % 딸기로 예상됩니다..


0214 1106   
데이터에 문제가 있는 거 같다 -> 데이터를 수정